In [51]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from pathlib import Path
import shutil

In [52]:
def removeIdentifierFile(path):
    for file in os.listdir(path):
        if file.find(".Identifier") != -1:
            os.remove(os.path.join(path, file))

In [53]:
import os, random

raw_imposters_folder = "imposters"
raw_valid_user_folder = "valid_user"

removeIdentifierFile(raw_imposters_folder)
removeIdentifierFile(raw_valid_user_folder)

img_count = 500 # img taken from each class

imposter_filenames = random.sample(os.listdir(raw_imposters_folder), img_count)
valid_user_filenames = random.sample(os.listdir(raw_valid_user_folder), img_count)

train_valid_user_folder = "train/valid_user"
train_imposters_folder = "train/imposters"

Path(train_valid_user_folder).mkdir(parents=True, exist_ok=True)
Path(train_imposters_folder).mkdir(parents=True, exist_ok=True)

test_valid_user_folder = "test/valid_user"
test_imposters_folder = "test/imposters"

Path(test_valid_user_folder).mkdir(parents=True, exist_ok=True)
Path(test_imposters_folder).mkdir(parents=True, exist_ok=True)


In [54]:
class_training_size = int(img_count * 0.9)

count = 0
for imposter_filename in imposter_filenames[:class_training_size]:
    shutil.copy(os.path.join(raw_imposters_folder, imposter_filename), os.path.join(train_imposters_folder, imposter_filename))
    count += 1

count = 0
for imposter_filename in imposter_filenames[class_training_size:]:
    shutil.copy(os.path.join(raw_imposters_folder, imposter_filename), os.path.join(test_imposters_folder, imposter_filename))


for valid_user_filename in valid_user_filenames[:class_training_size]:
    shutil.copy(os.path.join(raw_valid_user_folder, valid_user_filename), os.path.join(train_valid_user_folder, valid_user_filename))

for valid_user_filename in valid_user_filenames[class_training_size:]:
    shutil.copy(os.path.join(raw_valid_user_folder, valid_user_filename), os.path.join(test_valid_user_folder, valid_user_filename))

removeIdentifierFile(train_imposters_folder)
removeIdentifierFile(test_imposters_folder)
removeIdentifierFile(train_valid_user_folder)
removeIdentifierFile(test_valid_user_folder)


In [55]:
train = ImageDataGenerator(rescale=1/255)
test = ImageDataGenerator(rescale=1/255)

train_dataset = train.flow_from_directory("train",
    target_size=(150,150),
    batch_size = 32,
    class_mode = 'binary')
                                         
test_dataset = test.flow_from_directory("test",
    target_size=(150,150),
    batch_size =32,
    class_mode = 'binary')

Found 900 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


In [56]:
test_dataset.class_indices


{'imposters': 0, 'valid_user': 1}

In [57]:
model = keras.Sequential()

model.add(keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(keras.layers.MaxPool2D(2,2))

model.add(keras.layers.Conv2D(64,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(512,activation='relu'))

model.add(keras.layers.Dense(1,activation='sigmoid'))

In [58]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [59]:
model.fit_generator(train_dataset,
    steps_per_epoch = 250,
    epochs = 10,
    validation_data = test_dataset
)

model.save('classifier')

/tmp/ipykernel_1448/1593974248.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_dataset,


Epoch 1/10
 29/250 [==>...........................] - ETA: 1:55 - loss: 0.2633 - accuracy: 0.8722WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 2500 batches). You may need to use the repeat() function when building your dataset.


250/250 [==============================] - 19s 64ms/step - loss: 0.2633 - accuracy: 0.8722 - val_loss: 0.0053 - val_accuracy: 1.0000


INFO:tensorflow:Assets written to: classifier/assets


INFO:tensorflow:Assets written to: classifier/assets


In [60]:
def predictImage(filename):
    img1 = image.load_img(filename,target_size=(150,150))
    model = tf.keras.models.load_model('classifier')
 
    Y = image.img_to_array(img1)
    
    X = np.expand_dims(Y,axis=0)
    val = model.predict(X, verbose=0)
    return val[0][0]




In [61]:
test_imposter = "test/imposters"
test_valid_user = "test/valid_user"
correct_count = 0
wrong_count = 0
count = 0

for file in os.listdir(test_imposter):
    id = predictImage(os.path.join(test_imposter, file))
    count += 1
    if id == 0:
        correct_count += 1
        print(f"{file} correct {id}")
    else:
        wrong_count += 1
        print(f"{file} wrong {id}")


for file in os.listdir(test_valid_user):
    id = predictImage(os.path.join(test_valid_user, file))
    count += 1
    if id == 1:
        correct_count += 1
        print(f"{file} correct {id}")
    else:
        wrong_count += 1
        print(f"{file} wrong {id}")

5797.png correct 0.0
2489.png correct 0.0
5949.png correct 0.0
6284.png correct 0.0
3766.png correct 0.0
5525.png correct 0.0
5011.png correct 0.0
3222.png correct 0.0
5363.png correct 0.0
5195.png correct 0.0
2977.png correct 0.0
2491.png correct 0.0
0335.png correct 0.0
0775.png correct 0.0
1217.png correct 0.0
2395.png correct 0.0
0749.png correct 0.0
5889.png correct 0.0
3535.png correct 0.0
5741.png correct 0.0
5848.png correct 0.0
0604.png correct 0.0
4852.png correct 0.0
1123.png correct 0.0
6834.png correct 0.0
2881.png correct 0.0
6480.png correct 0.0
5868.png correct 0.0
5539.png correct 0.0
3575.png correct 0.0
3597.png correct 0.0
0092.png correct 0.0
6436.png correct 0.0
0111.png correct 0.0
3773.png correct 0.0
5775.png correct 0.0
4318.png correct 0.0
5565.png correct 0.0
2682.png correct 0.0
5103.png correct 0.0
3827.png correct 0.0
6689.png correct 0.0
6586.png correct 0.0
4829.png correct 0.0
3822.png correct 0.0
6283.png correct 0.0
0507.png correct 0.0
0657.png corr

In [62]:
print(correct_count / count)


1.0
